In [1]:
import json

import numpy as np

text=[]
f_read=open('体育.json', 'r',encoding='utf8',errors='ignore')
for line in f_read:
    line=line.replace('\\u0009','').replace('\\n','')
    obj=json.loads(line)
    sent=obj['contentClean']
    text.append(sent)

In [3]:
import jieba
processed_text=[]
for sent in text:
    processed_sent=jieba.cut(sent.strip(' '))
    processed_text.append(list(processed_sent))
print(processed_text[0])

Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\shulin\AppData\Local\Temp\jieba.cache
Loading model cost 1.090 seconds.
Prefix dict has been built successfully.


['远', '在', '土耳其', '打球', '的', '朱婷', '迎来', '自己', '的', '大', '日子', '，', '今年', '11', '月', '29', '日', '是', '这位', '中国女排', '当家', '球星', '的', '22', '岁', '生日', '。', '尽管', '在', '国外', '，', '但', '朱婷', '还是', '感受', '到', '了', '家乡', '的', '温暖', '，', '因为', '29', '日', '她', '有', '一场', '特别', '的', '生日会', '，', '腾讯', '体育', '也', '对', '这场', '生日会', '进行', '了', '全程', '直播', '。', ' ', ' ', ' ', ' ', '郎导', '携', '女儿', '录像', '送祝福', '，', '黄晓明', '成', '意外', '惊喜', ' ', ' ', '当地', '时间', '13', '点', '30', '分', '，', '朱婷', '的', '生日会', '正式', '开始', '。', '作为', '当天', '的', '绝对', '主角', '，', '朱婷', '结束', '了', '上午', '的', '训练', '匆匆', '赶来', '。', '她', '身穿', '运动服', '刚', '进入', '会场', '，', '参加', '生日会', '的', '球迷', '和', '记者', '就', '颇', '有', '默契', '地', '一起', '为', '朱婷', '高唱', '生日快乐', '，', '现场', '其乐融融', '。', '谦逊', '的', '朱婷', '开口', '第一句', '就是', '感谢', '，', '“', '这是', '第一次', '在', '国外', '过生日', '，', '很', '感动', '，', '大家', '特意', '从', '国内', '飞过来', '，', '让', '我', '在', '海外', '也', '有家', '的', '感觉', '。', '以后', '我要', '以', '更好', '的', '训练', '和', '比赛', '来', '回报', '大家

In [4]:
from gensim.models import word2vec
w_model = word2vec.Word2Vec(processed_text,hs=1,sg=0,min_count=1,window=3,size=100)

In [5]:
w_model.save('w_model')

In [6]:
from gensim.models import word2vec
w_model = word2vec.Word2Vec.load('w_model')
vocab=list(w_model.wv.vocab.keys())
print(vocab[:100])
vec_example = w_model.wv['日子']
print(vec_example)
sims = w_model.wv.most_similar('自己', topn=10)
print(sims)
print(w_model.wv.similarity('他们','自己'))

['远', '在', '土耳其', '打球', '的', '朱婷', '迎来', '自己', '大', '日子', '，', '今年', '11', '月', '29', '日', '是', '这位', '中国女排', '当家', '球星', '22', '岁', '生日', '。', '尽管', '国外', '但', '还是', '感受', '到', '了', '家乡', '温暖', '因为', '她', '有', '一场', '特别', '生日会', '腾讯', '体育', '也', '对', '这场', '进行', '全程', '直播', ' ', '郎导', '携', '女儿', '录像', '送祝福', '黄晓明', '成', '意外', '惊喜', '当地', '时间', '13', '点', '30', '分', '正式', '开始', '作为', '当天', '绝对', '主角', '结束', '上午', '训练', '匆匆', '赶来', '身穿', '运动服', '刚', '进入', '会场', '参加', '球迷', '和', '记者', '就', '颇', '默契', '地', '一起', '为', '高唱', '生日快乐', '现场', '其乐融融', '谦逊', '开口', '第一句', '就是', '感谢', '“']
[-0.10103221 -0.01037199  0.01891147  0.00424865  0.05575441 -0.11317477
 -0.12420414 -0.06976337  0.05644816 -0.03487032  0.05933174 -0.1391017
 -0.05675845 -0.01657037  0.03558956 -0.00299345 -0.00651229  0.139083
  0.03215731 -0.05618741 -0.00405105  0.04392483 -0.00611543 -0.18129271
  0.05867305 -0.06647798  0.01377796 -0.01618974 -0.02423334 -0.08508847
 -0.01409591 -0.00801949  0.10607313 -0.06380919 -0.02

In [26]:
import gensim
from gensim.models.doc2vec import Doc2Vec,LabeledSentence
train_text=[]
for i,sent in enumerate(processed_text):
    tagged_doc=gensim.models.doc2vec.TaggedDocument(sent,tags=[i])
    train_text.append(tagged_doc)
d_model=Doc2Vec(train_text,min_count=5,windows=3,vector_size=100,sample=0.001,nagetive=5)
d_model.train(train_text,total_examples=d_model.corpus_count,epochs=10)
d_model.save("doc2vec_model")
# train_text[1]

d_model= gensim.models.doc2vec.Doc2Vec.load("doc2vec_model")
text_vecs= d_model.docvecs.vectors_docs
print("专利向量的个数为",len(text_vecs))
v1 = d_model.infer_vector(['我们','是','中国','人','篮球'])
v2 = d_model.infer_vector(['我们','打','篮球'])
print(v1)
print(v2)

专利向量的个数为 500
[ 0.00156461  0.04865577 -0.04272801  0.0259273   0.10623727 -0.02747454
 -0.02723789  0.07509695 -0.01750571 -0.03977581 -0.02953502 -0.01654369
 -0.04880445 -0.06850068 -0.0409689   0.00194363 -0.01438847  0.03576699
  0.00056264 -0.0592368   0.01585501 -0.00864235  0.06447652 -0.08558173
  0.06467085 -0.01028275  0.0106414  -0.04119305 -0.00843612 -0.02825488
 -0.01079093  0.00911659 -0.00211041 -0.0098516   0.02603169  0.02409869
 -0.00093505  0.0303847  -0.06443315 -0.02199232  0.01579224  0.05904455
 -0.01124061  0.02156533 -0.00383119  0.03528729 -0.03397252  0.01196303
  0.04432366 -0.03892498  0.00886844  0.00540819  0.09449809  0.03565025
  0.02554592 -0.01841017  0.05303336 -0.03504122 -0.03142571 -0.01312629
 -0.02580624  0.0257521   0.04740356  0.03924007 -0.05981071  0.0389812
 -0.00335017  0.05012696  0.01075539 -0.02312602 -0.03223344 -0.01815176
 -0.00954434  0.05862164  0.02155799  0.02234483 -0.02237745  0.01070984
 -0.02765507 -0.02799721 -0.02455642 -0

In [3]:
import numpy as np
np.array(1).shape

()